In [ ]:
from textwrap import dedent
import requests
import json
import os
import re
from typing import TypedDict, Annotated
from langchain_ollama import OllamaLLM
from langgraph.graph import StateGraph, END

# Initialize LLM model
model = OllamaLLM(model="llama3.2")

class Tools:
    @staticmethod
    def search_internet(query, num_results=3):
        """Enhanced search with cost data extraction"""
        url = "https://google.serper.dev/search"
        headers = {
            'X-API-KEY': os.environ['SERPER_API_KEY'],
            'Content-Type': 'application/json'
        }
        response = requests.post(url, headers=headers, data=json.dumps({'q': query}))
        results = response.json().get('organic', [])
        
        if "cost" in query.lower() or "price" in query.lower():
            cost_data = []
            for r in results[:num_results]:
                snippet = r.get('snippet', '')
                prices = re.findall(r'(?:USD|€|£|¥|₹|₩|₽|฿|₴|₫|₪|元|CAD|AUD)\s*[\d,]+\.?\d*', snippet)
                if prices:
                    cost_data.append(f"From {r['title']} ({r['link']}): {', '.join(prices)}")
            return "\n".join(cost_data) if cost_data else "No explicit cost data found"
        
        return "\n".join(
            f"Title: {r['title']}\nLink: {r['link']}\nSnippet: {r.get('snippet', '')}\n"
            for r in results[:num_results]
        )

    @staticmethod
    def generate_with_llama(prompt):
        """Generate text using LLM with error handling"""
        try:
            response = model.generate([prompt])
            return response.generations[0][0].text
        except Exception as e:
            return f"LLM Error: {str(e)}"

class TripPlanningState(TypedDict):
    Country_Destination: str
    cities: list[str]
    date_range: str
    interests: str
    best_city: Annotated[str, "Selected destination"]
    city_info: Annotated[str, "Researched city details"]
    itinerary: Annotated[str, "Daily schedule"]
    cost_estimation: Annotated[str, "Detailed cost analysis"]
    report: Annotated[str, "Final compiled report"]

def select_best_city(state: TripPlanningState):
    """Research and select optimal destination"""
    research = "\n".join(
        f"Research for {city}:\n{Tools.search_internet(f'{city} {state["Country_Destination"]} tourism {state["interests"]}')}"
        for city in state['cities']
    )
    
    prompt = f"""
    Analyze these destinations in {state['Country_Destination']}:
    {research}
    
    Select best option for {state['interests']} considering:
    - Cultural relevance
    - Seasonal suitability
    - Accessibility
    """
    return {"best_city": Tools.generate_with_llama(prompt)}

def gather_city_info(state: TripPlanningState):
    """Compile comprehensive city guide"""
    search_data = Tools.search_internet(
        f"{state['best_city']} {state['Country_Destination']} {state['date_range']} {state['interests']} guide"
    )
    
    prompt = f"""
    Create travel guide for {state['best_city']}, {state['Country_Destination']} using:
    {search_data}
    
    Include:
    - Top attractions
    - {state['interests']}-specific activities 
    - Local customs
    - {state['date_range']} events
    """
    return {"city_info": Tools.generate_with_llama(prompt)}

def estimate_trip_costs(state: TripPlanningState):
    """Dedicated cost estimation node with prompt"""
    # Get raw price data
    hotel_data = Tools.search_internet(
        f"average hotel price in {state['best_city']} {state['Country_Destination']} {state['date_range']}"
    )
    food_data = Tools.search_internet(
        f"meal costs in {state['best_city']} for {state['interests']} travelers"
    )
    activity_data = Tools.search_internet(
        f"activity prices in {state['best_city']} for {state['interests']}"
    )
    
    prompt = f"""
    Analyze these cost data points for {state['best_city']}, {state['Country_Destination']}:
    
    ACCOMMODATION:
    {hotel_data}
    
    FOOD:
    {food_data}
    
    ACTIVITIES:
    {activity_data}
    
    Generate detailed cost estimation for {state['date_range']} including:
    1. Three budget tiers (budget/mid-range/luxury)
    2. Daily cost breakdown
    3. Potential unexpected expenses
    4. Best value recommendations
    5. Currency exchange advice if applicable
    """
    return {"cost_estimation": Tools.generate_with_llama(prompt)}

def create_itinerary(state: TripPlanningState):
    """Generate budget-aware itinerary"""
    prompt = f"""
    Create 7-day itinerary for {state['best_city']}, {state['Country_Destination']}:
    
    BUDGET CONTEXT:
    {state.get('cost_estimation', 'No specific budget constraints')}
    
    INTERESTS:
    {state['interests']}
    
    DATES:
    {state['date_range']}
    
    Include for each day:
    - Budget-friendly alternatives
    - Estimated activity costs
    - Free time suggestions
    """
    itinerary = Tools.generate_with_llama(prompt)
    return {"itinerary": itinerary}

def final_report(state: TripPlanningState):
    """Final report with integrated cost analysis"""
    report = f"""
    # COMPREHENSIVE TRAVEL PLAN
    ## Destination: {state['best_city']}, {state['Country_Destination']}
    ## Travel Period: {state['date_range']}
    
    ## Cost Estimation
    {state.get('cost_estimation', 'Cost analysis not available')}
    
    ## Destination Guide
    {state['city_info']}
    
    ## Daily Itinerary
    {state['itinerary']}
    
    ## Financial Tips
    {Tools.generate_with_llama(f"5 expert money management tips for traveling in {state['Country_Destination']}")}
    """
    return {"report": dedent(report)}

def create_workflow():
    """Build workflow with cost estimation node"""
    workflow = StateGraph(TripPlanningState)
    
    workflow.add_node("select_city", select_best_city)
    workflow.add_node("gather_info", gather_city_info)
    workflow.add_node("estimate_costs", estimate_trip_costs)
    workflow.add_node("plan_itinerary", create_itinerary)
    workflow.add_node("generate_report", final_report)
    
    workflow.add_edge("select_city", "gather_info")
    workflow.add_edge("gather_info", "estimate_costs")
    workflow.add_edge("estimate_costs", "plan_itinerary")
    workflow.add_edge("plan_itinerary", "generate_report")
    workflow.add_edge("generate_report", END)
    
    workflow.set_entry_point("select_city")
    return workflow.compile()

def main():
    print("🌍 Trip Planner with Cost Estimation")
    print("-----------------------------------")
    
    initial_state = {
        "Country_Destination": input("Which country are you visiting? ").strip(),
        "cities": [c.strip() for c in input("Cities to consider (comma-separated): ").split(",")],
        "date_range": input("Travel dates (e.g., 'July 15-22, 2025'): ").strip(),
        "interests": input("Your travel interests: ").strip(),
        "best_city": "",
        "city_info": "",
        "itinerary": "",
        "cost_estimation": "",
        "report": ""
    }
    
    app = create_workflow()
    result = app.invoke(initial_state)
    
    print("\n✈️ FINAL TRAVEL PLAN ✈️")
    print(result['report'])
    
    with open(f"{initial_state['Country_Destination']}_trip_plan2.txt", "w") as f:
        f.write(result['report'])

if __name__ == "__main__":
    main()

🌍 Trip Planner with Cost Estimation
-----------------------------------

✈️ FINAL TRAVEL PLAN ✈️

    # COMPREHENSIVE TRAVEL PLAN
    ## Destination: Based on the research, I recommend Sousse as the top destination for swimming and partying in Tunisia. Here's why:

1. **Cultural relevance**: While Tunisian culture may not be immediately associated with beach parties, many hotels in Sousse do offer foam parties or other events that cater to tourists. This suggests a willingness to adapt to Western-style entertainment.
2. **Seasonal suitability**: Sousse has a Mediterranean climate, making it suitable for swimming and outdoor activities year-round. The summer months (June to September) are peak tourist season, but even in the spring (April to May) and autumn (September to November), the weather is pleasant, with average highs around 22°C (72°F).
3. **Accessibility**: Sousse has a well-developed tourist infrastructure, with many hotels, restaurants, and attractions catering to internation